# Leduc Poker Interactive Environment

This notebook allows you to interactively play Leduc Poker and test different agents.

### Game Rules
- **Deck**: 6 Cards (Jack, Queen, King of Spades & Hearts)
- **Round 1 (Pre-flop)**: Each player gets 1 card. Betting occurs (Limit 2 chips).
- **Round 2 (Flop)**: 1 Community card is revealed. Betting occurs (Limit 4 chips).
- **Showdown**: If you pair the board card, you win. Otherwise, High Card wins.

In [ ]:
# Import our environment and agents
import sys
import os

# Ensure we can import from the current directory
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from leduc_env import LeducHoldemEnv
from agents import RandomAgent, HeuristicAgent

### Helper Function to Print Game State Nicely

In [ ]:
def print_state(state, player_id):
    card = state['card']
    board = state['board']
    history = state['history']
    pot = state['pot']
    
    card_str = f"{['J','Q','K'][card[0]]}{['s','h'][card[1]]}"
    board_str = f"{['J','Q','K'][board[0]]}{['s','h'][board[1]]}" if board else "Hidden"
    
    action_map = {0: 'Fold', 1: 'Check/Call', 2: 'Bet/Raise'}
    hist_str = [action_map.get(a, str(a)) for a in history]
    
    print(f"--- Player {player_id} View ---")
    print(f"Hole Card: {card_str}")
    print(f"Board:     {board_str}")
    print(f"Pot:       {pot}")
    print(f"History:   {hist_str}")
    print("----------------------")

### Interactive Play vs Heuristic Bot
Run this cell to play a hand against the bot!

In [ ]:
env = LeducHoldemEnv()
bot = HeuristicAgent()

# You are Player 0
state = env.reset()
print("NEW GAME STARTED")
done = False

while not done:
    current_player = env.active_player
    legal = env.get_legal_actions()
    
    if current_player == 0:
        # Your Turn
        print_state(state, 0)
        print(f"Legal Actions: {legal} (0=Fold, 1=Check/Call, 2=Bet/Raise)")
        try:
            action = int(input("Enter action: "))
            if action not in legal:
                print("Illegal action! Try again.")
                continue
        except ValueError:
            print("Invalid input.")
            continue
    else:
        # Bot Turn
        # Bot needs to loop until it picks a valid action (simple wrapper)
        # We reuse the hack from arena.py
        action = -1
        while action not in legal:
            action = bot.act(env.get_state(1))
        print(f">>> Bot plays: {action}")
            
    state, payoffs, done = env.step(action)

print("\nGAME OVER")
print(f"Payoffs: {payoffs}")
if payoffs[0] > 0:
    print("You Won!")
elif payoffs[0] < 0:
    print("You Lost!")
else:
    print("Tie!")

### Run a Simulation: Bot vs Random
Let's simulate 1000 hands to see the win rate.

In [ ]:
from arena import play_match
bot_heuristic = HeuristicAgent()
bot_random = RandomAgent()

print("Simulating 1000 hands...")
results = play_match(bot_heuristic, bot_random, 1000)
print(f"Results: {results}")
print(f"Heuristic Bot Win Rate: {results[0] / 1000} chips/hand")